# Clustering

In this notebook we will code and use different clustering techniques. The objective is to learn how to use them and understand the effects of different parameters. 

We will test:

- kMeans
- hierarchical clustering
- DBscan

In [0]:
# libraries 

import numpy as np
import pandas as pd
import random 

import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=4, suppress=True)

# Generate random data

We're going to generate random data normally distributed around three centers, with noise. Each cluster will have 200 points. We concatenate all three groups in a single dataframe. 

In [0]:
NPOINTSPERCLUSTER = 200

In [0]:
# Set three centers
center_1 = np.array([0,0])
center_2 = np.array([3,4])
center_3 = np.array([6,1])

# Generate random data around the three centers
data_1 = np.random.randn(NPOINTSPERCLUSTER, 2) + center_1
data_2 = np.random.randn(NPOINTSPERCLUSTER, 2) + center_2
data_3 = np.random.randn(NPOINTSPERCLUSTER, 2) + center_3

data = np.concatenate((data_1, data_2, data_3), axis = 0)
print(data.shape)

data[0:10]

In [0]:
plt.scatter(data[:,0], data[:,1], s=7); #s=size

Please, note that the data we have created does not have a class. It is just a set of points. However, we DO know that they come from different distribution and our objective is to find out them. 

# Implementation of K-Means

In this section we will implement kmeans algorithm. 


## Algorithm

```
clustering (data, K):
    Randomly initialize K cluster centroids (mu(1),..., mu(k))
    # or select K random points from data
    Repeat until convergence:
        # assign cluster
        for d in data:
            assign d to closest cluster centroid
        # recompute cluster centroid
        for k = 1 to K:
            mu(k) = mean of data points assigned to cluster k
            
```

### Exercise 1: complete the following code so that it follows k-means algorithm

- Assume data is a a (Npoints, dim) numpy array. 
- Hint: use `numpy.linalg.norm()` to compute euclidean distance
- Hint: `np.random.choice()` function may be useful

*15 min*

In [0]:
def kmeans (data, K, plot=False):
    N = data.shape[0]
    dim = data.shape[1]
    
    # generate K random centroids
    centroids = 0 #TO-DO
    
    # initialize vectors
    new_centroids = np.zeros((K, dim))
    distances = np.zeros((N, K))
    
    
    # repeat until convergence
    niter = 1
    while True:
        
        # compute distance of each point to cluster centroid
        for i in range(K):
            distances[:,i] = 0 #TO-DO
         
        
        # assign points to closest centroid
        clusters = 0 #TO-DO
        
        # recompute clusters' centroids
        # Calculate mean for every cluster and update the center
        for i in range(K):
            new_centroids[i] = 0 #TO-DO
        
        # compute if there is any variation
        if np.array_equal(centroids, new_centroids):
            break
            
        centroids = new_centroids.copy()

        if plot:
            # solution
            plt.scatter(data[:,0], data[:,1], c=clusters, cmap='plasma', linewidths=0);
            for k in range(K):
                plt.scatter(centroids[k,0], centroids[k, 1], s=100, marker='D', color='red')
            plt.show()

        niter+=1
        
    return clusters, centroids    

In [0]:
# let's try our algorithm
K = 3
clusters, centroids = kmeans(data, K)

In [0]:
plt.scatter(data[:,0], data[:,1], c=clusters, cmap="plasma", linewidths=0);

Now we are going to compare the original 'cluster' where each point comes from, with the asigned cluster. In order to do that, we just create a vector with the original class (color) and use that to plot. 

Please, recall that this is something we can do here because we're creating a synthetic dataset, but normally we won't be able to do it, since we don't know how the data has been generated. 

In [0]:
# comparison to original distribution where data came from
original = np.concatenate((np.repeat(0, NPOINTSPERCLUSTER), 
                         np.repeat(1, NPOINTSPERCLUSTER), 
                         np.repeat(2, NPOINTSPERCLUSTER)))
plt.scatter(data[:,0], data[:,1], c=original, cmap="plasma", linewidths=0);

In [0]:
# comparison to original distribution where data came from
# we create a df with the clusters and compute the 'confussion' matrix

df = pd.DataFrame({'original' : original, 'kmeans': clusters})
df_g = df.groupby(['original', 'kmeans']).size().reset_index(name='n')\
    .pivot(index='original', columns='kmeans', values='n').fillna(0)
df_g

Notice that class labels (kmeans) may not agree with original class number.

In [0]:
from sklearn.metrics import accuracy_score

mapping = {i:v for (i,v) in df_g.idxmax(axis=1).items()}
print(mapping)
accuracy_score([mapping[o] for o in original], clusters)

In [0]:
K = 3
clusters, centroids = kmeans(data, K, plot=True)

### Exercise 2: Take some time to play with different number of (original) distributions and clusters and see the effect when number of clusters does not match true data

15 minutes. 

In [0]:
# Set three centers
center_1 = np.array([0,0])
center_2 = ...
center_3 = ...
...

# Generate random data around the three centers
data_1 = np.random.randn(NPOINTSPERCLUSTER, 2) + center_1
data_2 = ...
... 

data = np.concatenate((data_1, data_2, ...), axis = 0)

k = ...
kmeans(data, k)

## Computing how good the cluster partition is

Remember SSE (Sum of Squared Error):
$$
SSE = \sum_{i=1}^N (x_i - C_{(X_i)})^2
$$
where $C_{(X_i)}$ represents the cluster centroid of $X_i$.



### Exercise 3: complete SSE function and compute clustering metrics for different number of clusters

In [0]:
def sse(data, clusters, centroids):
    return 0 # TO-DO

In [0]:
ks = range(2, 20)
sse_errors = np.zeros(len(ks))

for i, k in enumerate(ks):
    clusters_, centroids_ = kmeans(data, k)
    sse_errors[i] = sse(data, clusters_, centroids_)
    print(k, sse_errors[i])
    
    #if np.isnan(sse_errors[i]):
    #    print(clusters_)
    #    print(centroids_)

In [0]:
plt.plot(ks, sse_errors, 'o-');
plt.xlim(min(ks), max(ks));

# scikit-learn K-means

Now we are going to use scikit-learn library for our exercise

KMeans works as other models in sklearn:

- define the model (and parameters)
- fit the model on training dataset
- apply the fitted model on another dataset (can be the same dataset)

In [0]:
from sklearn.cluster import KMeans

In [0]:
# inspect help
?KMeans

In [0]:
# define the model
model = KMeans(n_clusters = 3)

In [0]:
# fit the model on training data
model = model.fit(data)

In [0]:
# press tab to see available methods
#model. #press tab
print(model.cluster_centers_)
print(model.labels_)
print(model.inertia_) # sse

In [0]:
# check inertia is our sse defined function
abs(model.inertia_ - sse(data, model.labels_, model.cluster_centers_)) < 0.01

In [0]:
# apply the fitted model 
# if applied to the same data, we get model.labels_
clusters_sk = model.predict(data)
all(model.labels_ == clusters_sk)

In [0]:
clusters_sk

In [0]:
def plot_clustering(data, clusters, centroids = []):
    
    K_ = len(set(clusters))
    
    plt.figure(figsize=(8,4))
    plt.scatter(data[:,0], data[:,1], c=clusters, cmap="plasma", linewidths=0)

    if centroids != []:
        for k in range(K_):
            plt.scatter(centroids[k,0], centroids[k, 1], s=100, marker='D', color='red')
        
    plt.show()

In [0]:
plot_clustering(data, clusters_sk)

In [0]:
# Centroid values
centroids_sk = model.cluster_centers_
centroids_sk

In [0]:
plot_clustering(data, clusters_sk, centroids_sk)

In [0]:
# compare with our implementation

print(centroids)
print(centroids_sk)

#plot_clustering(data, clusters, centroids)
#plot_clustering(data, clusters_sk, centroids_sk)

# Hierarchical Clustering

In this section we will learn how to apply Hierarchical Clustering in Python. 

We will use `scipy` package:

- `linkage` function computes the distance matrix between the points
- `dendrogram` function plots the dendrogram using the distances
- `fcluster` function performs a clustering assignment according to different parameters

![texto alternativo](https://drive.google.com/uc?id=1MdZuOQ-SNcglxN0IXEobgrOTZR7NuaM-)

In [0]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

In [0]:
?linkage

In [0]:
Z = linkage(data, 'ward') #'single', 'ward', ...

In [0]:
plt.figure(figsize=(14, 7))
dendrogram(Z)
plt.show()

Now let's understand how the algorithm works. 

It follows an agglomerative approach, so *close* points are merged. 

`linkage` returns how points are merged at each iteration. The output is: [`id_node_1`, `id_node_2`, `distance`, `number_of_points_in_group`]

In [0]:
Z[0:20]

In [0]:
# indices
np.where(Z[:,0] > data.shape[0])[0:20]

In [0]:
Z[Z[:,0] > data.shape[0]][0:20]

### Exercise 4: dendrogram plotting options

Investigate dendrogram plotting options and play with them.

- p:
- truncate_mode: 'lastp', 'level'
- color_threshold
- orientation
- count_sort: False, 'ascending'/True, 'descendent'
- distance_sort: False, 'ascending'/True, 'descendent'
- show_leaf_counts: boolean (True)
- show_contracted: boolean (False)
- above_threshold_color = 'b'

*10 min*

In [0]:
# exercise
plt.figure(figsize=(14, 4))
dendrogram(Z, ...)
plt.show()

In [0]:
# exercise
plt.figure(figsize=(14, 4))
dendrogram(Z, ...)
plt.show()

In [0]:
# exercise
plt.figure(figsize=(14, 4))
dendrogram(Z, ...)
plt.show()

## Getting the cluster partition

In order to assign a cluster to each sample we first need to set the cut_off distance. We will visually explore what this value is and use it for partitioning. 

In [0]:
?dendrogram

In [0]:
dendrogram(Z, color_threshold = 25)
plt.show()

In [0]:
from scipy.cluster.hierarchy import fcluster

?fcluster

In [0]:
cut_distance = 25
clusters_hc = fcluster(Z, t = cut_distance, criterion='distance')
np.unique(clusters_hc, return_counts = True)

In [0]:
clusters_hc

### Exercise 5: change cut_distance and see how the number of clusters change

It should match the dendrogram plotting

In [0]:
# exercise
cut_distance = 


### Exercise 6 (home): change `method` parameter in the linkage function and observe the results

In [0]:
# To-Do
linked = linkage(data, 'to-do') #'single', 'ward', ...
plt.figure(figsize=(10, 5))
dendrogram(linked)
plt.show()

In [0]:
dendrogram(linked, color_threshold=cut_distance)
plt.show()

#### AgglomerativeClustering in scikit-learn

In order to perform the clustering partition, we can also use `AgglomerativeClustering` from `sklearn` once we have selected the desired number of clusters. 

In [0]:
from sklearn.cluster import AgglomerativeClustering

# define the model
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')  

# fit data and predict 
clusters = cluster.fit_predict(data)

In [0]:
plot_clustering(data, clusters)

### Comparison kmeans and hierarchical

Now let's compare the result of both algorithms when the number of clusters is not optimal

In [0]:
NCLUS = 5

km = KMeans(n_clusters=NCLUS)
clusters_km = km.fit_predict(data)
plot_clustering(data, clusters_km, km.cluster_centers_)

hier = AgglomerativeClustering(n_clusters=NCLUS, affinity='euclidean', linkage='ward')
clusters_hier = hier.fit_predict(data)
plot_clustering(data, clusters_hier)

# DBSCAN

In this section we will learn how DBscan algorithm works and what's the effect of its parameters in the clustering result. 

In [0]:
from sklearn.cluster import DBSCAN

In [0]:
dbs = DBSCAN(eps=0.5, min_samples=5)
dbs = dbs.fit(data)

In [0]:
dbs.labels_[0:50]

In [0]:
id_clusters = np.unique(dbs.labels_)
print('Found {} clusters'.format(len(id_clusters)))
np.unique(dbs.labels_, return_counts=True)

In [0]:
plt.scatter(data[:,0], data[:,1], c=dbs.labels_, cmap="plasma", linewidths=0);

### Exercise 7: play with the parameters `eps` and `min_samples` and see how it affects the clustering partition
*5 min*

In [0]:
# exercise
dbs = DBSCAN(eps=..., min_samples=...)
dbs = dbs.fit(data)
plot_clustering(data, dbs.labels_)

# New data : non-convex datasets

Now we will apply DBscan on non convex data to see the differences with k-means. We will also learn how to load already-predefined datasets from `sklearn`

In [0]:
from sklearn import datasets

nsamples = 1000

In [0]:
# make_blobs
# this creates 'centers' circles
# we will change the covariance so that the clusters become ellipses

X, y = datasets.make_blobs(random_state=170, n_samples=nsamples, centers = 5)
transformation = np.random.RandomState(74).normal(size=(2, 2))
X = np.dot(X, transformation)

# plot
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel("Feature 0")
plt.ylabel("Feature 1")
plt.show()

Compare DBscan against Kmeans on that data

In [0]:
dbscan_model = DBSCAN(eps=0.4, min_samples=5)
dbscan_model = dbscan_model.fit(X)

kmeans_model = KMeans(n_clusters = 5)
kmeans_model = kmeans_model.fit(X)

plot_clustering(X, dbscan_model.labels_)
plot_clustering(X, kmeans_model.labels_, kmeans_model.cluster_centers_)

In [0]:
# Exercise: test other datasets
# datasets.make_circles
# datasets.make_moons
# datasets.make_s_curve (Hint: use dimensions 0 and 2 of the generated dataset)
# datasets.make_swiss_roll (Hint: use dimensions 0 and 2 of the generated dataset)

In [0]:
# make_circles
# ...

In [0]:
# make_mooons
# ...

In [0]:
# make_s_curve
# ... 

In [0]:
# make_swiss_roll
# ...

Some code and ideas are based on:

- https://mubaris.com/posts/kmeans-clustering/
- https://www.kaggle.com/andyxie/k-means-clustering-implementation-in-python
- https://stackabuse.com/hierarchical-clustering-with-python-and-scikit-learn/


### End.